Last Updated 11/12/2016

The aim here is to generate the angle between IMAGE and its apogee vector over the course of the mission.

In [1]:
import datetime          as dt
import pickle
import numpy             as np
import scipy             as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
work_path = 'c:/users/cschiff/Documents/GitHub/FAST-and-IMAGE/'
home_path = 'c:/users/Conrad/Documents/GitHub/FAST-and-IMAGE/'
IMAGE_eph = open(home_path+'IMAGE_full_eph.eph','r')

In [3]:
IMAGE_date_format = '%d-%m-%Y %H:%M:%S.%f'

In [4]:
IMAGE_blob = IMAGE_eph.readlines()
IMAGE_eph.close()

In [26]:
N      = len(IMAGE_blob)
Epochs = [0]*N
States = np.zeros((N,6))
Ls     = np.zeros((N,3))
nEccs  = np.zeros((N,3))
aTAs   = np.zeros((N))

In [27]:
mu      = 398600.4414  #gravitational constant of the Earth
counter = 0
for line in IMAGE_blob:
    record              = line.strip('\n\r').split()
    epoch               = dt.datetime.strptime(record[0]+' '+record[1],IMAGE_date_format)
    Epochs[counter]     = epoch
    vx, vy, vz, x, y, z = map(float,record[2:])
    r                   = np.array([x,y,z])
    r_norm              = np.sqrt(r.dot(r))
    v                   = np.array([vx,vy,vz])
    v_norm              = np.sqrt(v.dot(v))
    L                   = np.array([y*vz-z*vy,z*vx-x*vz,x*vy-y*vx])
    Lx, Ly, Lz          = L
    L_norm              = np.sqrt(L.dot(L))
    v_cross_L           = np.array([vy*Lz-vz*Ly,vz*Lx-vx*Lz,vx*Ly-vy*Lx])
    n_ecc               = mu*r/r_norm - v_cross_L 
    n_ecc_norm          = np.sqrt(n_ecc.dot(n_ecc))
    n_nu                = np.arccos(r.dot(n_ecc)/r_norm/n_ecc_norm)*180/np.pi
    States[counter,:]   = np.array([x,y,z,vx,vy,vz])
    Ls[counter,:]       = L
    nEccs[counter,:]    = n_ecc
    aTAs[counter]       = n_nu
    counter             = counter + 1
Epochs = np.asarray(Epochs)

Want to perform a spot-check of the computation at some chosen time.  Compared against FreeFlyer with the following state:

- Epoch: 27-03-2000 00:20:55.000
- x:     25706.
- y:     5814.0
- z:     -3150.6.
- vx:    3.27455
- vy:    -0.740804
- vz:    -2.32263

- TA from FreeFlyer:  227.04

In [28]:
time_index = 10

In [29]:
Epochs[time_index]

datetime.datetime(2000, 3, 27, 0, 20, 55)

In [30]:
States[time_index,:]

array([  2.57060000e+04,   5.81400000e+03,  -3.15060000e+03,
        -3.27455000e+00,  -7.40804000e-01,  -2.32263000e+00])

In [31]:
Ls[time_index,:]

array([ -1.58377479e+04,   7.00223240e+04,  -4.87392400e+00])

In [32]:
nEccs[time_index,:]

array([ 223392.48750554,   50540.7108885 ,  193711.08936543])

In [35]:
aTAs[time_index]

47.040130385546021

In [36]:
aTAs[time_index]+180

227.04013038554604

Since everything passed, pickle away the results so that we never have to do that again (or at least only infrequently)

In [37]:
pickle.dump((Epochs,aTAs),open(home_path+'IMAGE_states_pickled.p','wb+'))

In [14]:
good = np.where(abs(aTAs) <= 30)

In [38]:
Epochs[good]

array([datetime.datetime(2000, 3, 27, 4, 36, 55),
       datetime.datetime(2000, 3, 27, 4, 38, 55),
       datetime.datetime(2000, 3, 27, 4, 40, 55), ...,
       datetime.datetime(2005, 12, 19, 20, 4),
       datetime.datetime(2005, 12, 19, 20, 6),
       datetime.datetime(2005, 12, 19, 20, 8)], dtype=object)